In [ ]:
from prompt_builder import prompt, parser


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import pandas as pd
df = pd.read_csv('langchain_structured_profiles.csv')
memory = df.iloc[0].to_json()

'{"name":"Anika Sharma","age":15,"class_level":10,"syllabus":"CBSE","interests":"[\'Coding\', \'Astronomy\', \'Debate\']","personality":"[\'Curious\', \'Creative\', \'Determined\']","hobbies":"[\'Reading\', \'Drawing\', \'Hiking\']","learning_style":"[\'Visual\', \'Kinesthetic\']","pattern_of_learning":"[\'Hands-on\', \'Group work\']","study_routine_start":"18:00","study_routine_end":"21:00","preferred_subjects":"[\'Math\', \'Science\', \'English\']","struggles_with":"[\'History\', \'Physics\']","strengths":"[\'Problem-solving\', \'Communication\']","challenges":"[\'Public speaking\', \'Time management\']","academic_progress":"Steady improvement in Math and Science.","marks_last_year_final":"{\'Math\': 92, \'Science\': 88, \'English\': 85, \'History\': 70, \'Physics\': 75}","marks_last_internals":"{\'Math\': 95, \'Science\': 90, \'English\': 88, \'History\': 75, \'Physics\': 80}","emotional_traits":"Generally positive and resilient, seeks clarification when needed.","motivation_style":

In [16]:
import os
from google import genai
from langchain import PromptTemplate

# 1) Install & import
# pip install google-genai

# 2) Configure (picks up GOOGLE_API_KEY from env)
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

# 4) Helper to call Gemini and return usage + JSON
def call_gemini_and_count(memory_block: str):
    prompt_text = prompt.format(memory_block=memory_block)

    # 4a) Send prompt
    resp = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt_text,
    )

    # 4b) Exact token metadata
    usage = resp.usage_metadata
    n_prompt    = usage.prompt_token_count
    n_completion = usage.candidates_token_count
    n_total      = usage.total_token_count

    # 4c) Generated JSON
    profile_json = resp.text

    return {
        "prompt_tokens":     n_prompt,
        "completion_tokens": n_completion,
        "total_tokens":      n_total,
        "profile_json":      profile_json,
    }

# 5) Pilot calls
res1 = call_gemini_and_count(memory_block="")            # i=1
first_profile = res1["profile_json"]
res2 = call_gemini_and_count(memory_block=first_profile) # i=2

# 6) Infer blocks
n_prompt      = res1["prompt_tokens"]
n_profile_in  = res2["prompt_tokens"] - n_prompt
n_profile_out = res2["completion_tokens"]

print(f"n_prompt={n_prompt}, n_profile_in={n_profile_in}, n_profile_out={n_profile_out}")

# 7) Extrapolate for N profiles
N = 1000
# sum_{i=1..N} [n_prompt + (i-1)*n_profile_in + n_profile_out]
total = N*(n_prompt + n_profile_out) + n_profile_in*(N*(N-1)//2)
print(f"Estimated total tokens for {N} profiles: {total:,}")


n_prompt=2071, n_profile_in=583, n_profile_out=548
Estimated total tokens for 1000 profiles: 293,827,500


In [17]:
res1['profile_json']

'{"name": "Anika Sharma", "age": 15, "class_level": 10, "syllabus": "CBSE", "interests": ["Coding", "Singing", "Dancing"], "personality": ["Creative", "Determined", "Friendly"], "hobbies": ["Drawing", "Reading", "Hiking"], "learning_style": ["Visual", "Kinesthetic"], "pattern_of_learning": ["Group Study", "Hands-on activities"], "study_routine_start": "17:00", "study_routine_end": "21:00", "preferred_subjects": ["Mathematics", "Science", "English"], "struggles_with": ["History", "Chemistry"], "strengths": ["Problem-solving", "Communication", "Teamwork"], "challenges": ["Public speaking", "Time management"], "academic_progress": "Anika has shown consistent improvement in her academic performance throughout the year.", "marks_last_year_final": {"Mathematics": 85, "Science": 88, "English": 92, "History": 70, "Chemistry": 75}, "marks_last_internals": {"Mathematics": 90, "Science": 92, "English": 95, "History": 75, "Chemistry": 80}, "emotional_traits": "Anika is generally positive and resil

In [18]:
parser.parse(res1['profile_json'])

StudentProfile(name='Anika Sharma', age=15, class_level=10, syllabus='CBSE', interests=['Coding', 'Singing', 'Dancing'], personality=['Creative', 'Determined', 'Friendly'], hobbies=['Drawing', 'Reading', 'Hiking'], learning_style=['Visual', 'Kinesthetic'], pattern_of_learning=['Group Study', 'Hands-on activities'], study_routine_start='17:00', study_routine_end='21:00', preferred_subjects=['Mathematics', 'Science', 'English'], struggles_with=['History', 'Chemistry'], strengths=['Problem-solving', 'Communication', 'Teamwork'], challenges=['Public speaking', 'Time management'], academic_progress='Anika has shown consistent improvement in her academic performance throughout the year.', marks_last_year_final={'Mathematics': 85, 'Science': 88, 'English': 92, 'History': 70, 'Chemistry': 75}, marks_last_internals={'Mathematics': 90, 'Science': 92, 'English': 95, 'History': 75, 'Chemistry': 80}, emotional_traits='Anika is generally positive and resilient, but can get discouraged by setbacks.',